# Models Description Tables
Here we generate a table that summarizes the model properties.

In [1]:
from cana.boolean_network import BooleanNetwork
import cubewalkers as cw

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
plt.rcParams.update({"font.size":20})

In [2]:
models_dir = "corrected_models" # only for corrected_models

QCFNS = pd.read_csv(f"data/{models_dir}/quasicoherence_fuzzy_sourceless.csv")
DC = pd.read_csv(f"data/{models_dir}/derrida_coefficients.csv")
QC = pd.read_csv("data/model_categories.csv")

In [3]:
QC = QC.merge(QCFNS)
QC = QC.merge(DC)

QC.columns = ["name", "category", "PMID",
              "SFQCNS", "AFQCNS",
              "SDC", "ADC", 
              "SDCNS", "ADCNS"]


N = []
NS = []
K = []
for name in sorted(QC["name"]):
    #print(name)
    with open(f"models/{models_dir}/{name}.txt") as f:
        model = cw.Model(f.read())
    # Translate to CANA readable
    string_rules = cw.conversions.cpp2bnet(model.rules)
    string_rules = string_rules.replace(",\t", "*= ")
    string_rules = string_rules.replace("!", "not ")
    string_rules = string_rules.replace("&", "and")
    string_rules = string_rules.replace("|", "or")
    
    # Load to CANA
    BN = BooleanNetwork.from_string_boolean(string_rules)
    
    constants = []
    k = 0
    for n in BN.nodes:
        k += n.k
        if n.k == 1 and n.inputs[0] == n.id:
            constants.append(n.id)
    k /= BN.Nnodes
    
    N.append(BN.Nnodes)
    NS.append(len(constants))
    K.append(k)
    

NDF = pd.DataFrame({
    "name": sorted(QC["name"]),
    "N": N,
    "NS": NS,
    "avgK": K,
})

QC=QC.merge(NDF)

# Fragility = 
# long-term difference in node avg. values after initial 1-node perturbation
QC["AFNS"] = (1-QC["AFQCNS"])*(QC["N"]-QC["NS"])
QC["SFNS"] = (1-QC["SFQCNS"])*(QC["N"]-QC["NS"])

QC = QC[["name", "category", "PMID",
         "N", "NS", "avgK",
         "SDC", "ADC", "SDCNS", "ADCNS",
         "SFQCNS", "AFQCNS", "SFNS", "AFNS"]]

#QC.sort_values(by=["name"])
QC.to_csv("data/tables/LatexTableSource.csv")

In [4]:
QC = pd.read_csv("data/tables/LatexTableSource.csv")

# Stress/Damage and Homeostasis = SDH

for category in QC["category"].unique():
    df = QC[QC.category == category]
    df.loc[:,"name"] = df["name"].str.replace("&", "\&")
    if category == "Signal Transduction (Stress/Damage and Homeostasis)":
        category = "Signal Transduction (SDH)"
    df.to_latex(buf=f"data/tables/{category}.tex", columns=["name", "PMID", "N", "NS", "avgK", "SDC", "AFNS"], index=False, 
                column_format="|p{180pt}||p{40pt}|p{25pt}|p{25pt}|p{40pt}|p{25pt}|p{25pt}|", float_format="%.4f",
                caption=category + " models", label="tab:"+category.replace(" ", "_"))